<a href="https://colab.research.google.com/github/fralfaro/AdvancedAnalyticsLabs/blob/master/docs/notebooks/python/Lab_5_WoE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Weight of Evidence Transformation

In this lab we will apply a Weight of Evidence transformation to our data. The idea is to:

- Split the data into a train/test set.
- Generate a relevant set of cuts to our data.
- Calculate the WoE for each variable.
- Save the data we just created.

We are assuming we have already cleaned the date of outliers and null values.

In order to do this we will use the fantastic  [```scorecardpy```](https://github.com/ShichenXie/scorecardpy) Python package. First we need to install it, as it is not a standard package.

We use the OS python software ```pip``` for this.

In [ ]:
!pip install git+https://github.com/CBravoR/scorecardpy

Now we import the data to use. I have created a clean version of the dataset from our last lab.

---



In [ ]:
import pandas as pd
!gdown 'https://drive.google.com/uc?id=1-RiFAF4zU27N9MnoSYUlNuqFhR3VcuWs'
bankloan_data = pd.read_pickle('BankloanClean.pkl')
bankloan_data.describe()

In [ ]:
sum(bankloan_data.Default==0)

In [ ]:
sum(bankloan_data.Default==1)

In [ ]:
bankloan_data.isnull().any()

In [ ]:
bankloan_data.OthDebt.fillna(value = 0, inplace=True)
bankloan_data.fillna(bankloan_data.median(), inplace=True)

It is always a good idea to create new variables to extract information from our models. For example, the following variable improves OthDebt.

In [ ]:
bankloan_data['OthDebtRatio'] = bankloan_data['OthDebt'] / bankloan_data['Income']

## Binning

The first step is to properly bin the data. Usually, we will run a tree and manually adjust those cases that do not follow a logical pattern.

However, as calculating WoE means we need to use the objective variable, we need to first create a train and test split. The scorecard package comes with a function to do so easily,  ```split_df```, which takes as an argument the ratio and the seed.

**Note: A random seed is used to generate a random split that will be reproducible (is there such as thing as randomness in a computer?). I expect for all coursework for you to use your student ID.**

In [ ]:
import scorecardpy as sc
import numpy as np

In [ ]:
# Split in train and test BEFORE we apply WoE
# Use your Student ID as seed!

train, test = sc.split_df(bankloan_data.iloc[:,1:],
                          y='Default',
                          ratio=0.7,
                          seed=20190227).values()

In [ ]:
test.describe()

In [ ]:
train.describe()

Now we can bin the variables. The function  ```woebin``` will do this automatically for us. It will use trees sequentially given the constraints we decide. It is good practice to not leave less than 5% of cases in each bin, and I am using 20 starting bins.

**Tip:  For larger datasets, use a relatively large number of bins (50 to 100), for smaller ones, use less.**

In [ ]:
bins = sc.woebin(train, y='Default',
                 min_perc_fine_bin=0.02, # How many bins to cut initially into
                 min_perc_coarse_bin=0.05,  # Minimum percentage per final bin
                 stop_limit=0.02, # Minimum information value
                 max_num_bin=8, # Maximum number of bins
                 method='tree'
                 )

Now we can plot the results. We need to be able to explain the results for each variable. We should be able to explain every trend. They do not need to necessarilly be linear, we just need a good explanation for the trend.

**If you cannot explain the trend you need to adjust the bins.**

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

sc.woebin_plot(bins)

In [ ]:
bins

## Manual adjustment

In this case, Creddebt and Income don't follow an explainable trend, so we need to make manual adjustments.

Note that OthDebt has an IV of 0.0927. **The output of the binning function is the best possible binning from a statistical point of view. Changing it will only make the IV go down.** As the IV of OthDebt is really close to 1, you need to make a call. How many variables do you have available? If plenty, then you can drop the variable, otherwise, you should leave it and see if this can be sorted out later.

Now we need to adjust Creddebt and Income. We can make this interactively within the package with the excelent function ```woebin_adj``` that allows us to make adjustments one by one, only for those variables with irregular trends (this can be changed in the options). We start by invoking the function, note the bar at the end.

In [ ]:
breaks_adj = sc.woebin_adj(train, "Default", bins, adj_all_var=True)

For Creddebt we need to enter the following breaks:

```1.0,6.0```

For OthDebt we enter

```0.4, 1.6, 3.2, 11.4```

For Income, we enter the following breaks:

```30.0,40.0,80.0,140.0```

And for OthDebtRatio we enter these ones:

```0.01,0.05,0.07,0.09,0.14```

The variables are now much better behaved. Play around with the breaks to see if you can get to better solutions.  OthDebt is a tricky variable: Its IV is 0.13 in the original (incorrect) binning, but the corrected one is (really slightly) below 0.1 so it would be discarded. Should we with a variable that close to 1? We could play around a bit more to see if we can get it to be rational.

Now that we are happy with the binnings, **we need to apply it to both of our datasets**.

In [ ]:
breaks_adj

In [ ]:
bins_adj = sc.woebin(train, y="Default", breaks_list=breaks_adj) # Apply new cuts
train_woe = sc.woebin_ply(train, bins_adj) # Calculate WoE dataset (train)
test_woe = sc.woebin_ply(test, bins_adj) # Calculate WoE dataset (test)

In [ ]:
train_woe.head()

The ```bins``` object is a dictionary. You can get each WoE table by calling the function ```get``` with the name of the variable as an argument.

In [ ]:
bins.get('MonthlyLoad')

You can see the adjusted breaks by calling the object. It is a good idea to save it somewhere as it saves time if you need to recalculate everything again (see next lab).

In [ ]:
breaks_adj

## Information Value Filtering

Now we can check the information value of our variables and remove those who are not predictive. We use the function ```iv```. In general:

- $IV < 0.02$: No predictive ability, remove.
- $0.02 \le IV < 0.1$: Small predictive ability, suggest to remove.
- $0.1 \le IV < 0.3$: Medium predictive ability, leave.
- $0.3 \le IV < 1$: Good predictive ability, leave.
- $1 \le IV $: Strong predictive ability. Suspicious variable. Study if error in calculation (i.e. WoE leaves a category with 100% goods or bads) or if variable is capturing future information.

In [ ]:
# Calculate variable importance
iv_df = pd.DataFrame(sc.iv(train_woe, 'Default'))
iv_df

In [ ]:
# Plot variable importance
sns.barplot(x="variable", y="info_value", data=iv_df)
plt.xticks(rotation=90)
plt.show()

As we can see, Education, and OthDebt are below the threshold. Income and OthDebt are a bit of an odd ones as they have a very close to one IV. **I recommend you leave Income as OthDebt was not explainable. Step 2 of variable selection can get rid of them if necessary**. We can easily remove the other variable manually.

In [ ]:
# Check column order.
train_woe.columns

In [ ]:
# Create range of accepted variables
accepted_range = np.r_[0:3,4:7,8:10] # Note the last in each subrange is not used
train_woe = train_woe.iloc[:, accepted_range]
test_woe = test_woe.iloc[:, accepted_range]
train_woe.head()

# Alternative
# train_woe.drop(axis = 0, index = ['Education_woe', any others], inplace = True)

In [ ]:
test_woe.head()

## Final correlation analysis

Now we need to do a final correlation analysis. WoE might introduce correlation effects which were originally not present. For this, we can use seaborn and the corr function in numpy.

In [ ]:
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Compute the correlation matrix
corr = train_woe.corr()
corr = np.abs(corr)

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

# Show the plot
plt.show()

In [ ]:
corr

There are a few variables that we need to key an eye on. First is Age and Address, with a correlation of 0.85. We can remove them at this stage, but let's keep an eye on the coefficients these variables give. In a logistic regression with WoE transformation, **all coefficients should be positive**, otherwise there are correlations effects at play.

In [ ]:
corr

## Saving the results.

Now we are ready to apply models! We have our train and test datasets ready. We can now save the csv into our local file system or our Google Drive. In the latter case, the process is a bit complicated, as it requires us to connect our accounts. The detailed instructions are [here](https://colab.research.google.com/notebooks/io.ipynb#scrollTo=vz-jH8T_Uk2c).

We will download the data to our own hard drive. First we need to save our data as csv, using the function [```to_csv```](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html).

In [ ]:
train_woe.drop(columns=['Address_woe', 'OthDebtRatio_woe'], inplace=True)
test_woe.drop(columns=['Address_woe', 'OthDebtRatio_woe'], inplace=True)
train_woe.to_csv("train_woe.csv", index = False)
test_woe.to_csv("test_woe.csv", index = False)
bankloan_data.to_pickle('BankloanCleanNewVars.pkl')
!ls # Linux commands to check what files are in the computer.

Now you can download this data using the sidebar.